In [2]:
from pathlib import Path
import os
path = Path().absolute()

if path.name =="GP":
    os.chdir(path.parent)

BASEDIR = Path().absolute()
BASEDIR

PosixPath('/home/taerim/PRIMNET-V2/code')

In [6]:
from utils.args import read_ARGS
configs = "PRIMNET/FINGER.py"
args = read_ARGS((BASEDIR/'configs'/configs).absolute())
args

ARGS(MODEL='PRIMNET', EVEN_JOINTS=True, WANDB=True, pname='PRIMNET_v2.1', runname='FINGER', DATASET='FINGER', TPOSE=((0, 0, 0.12),), LOAD_WEIGHTPATH=None, SAVE_PERIOD=1, TEST_PERIOD=1, EVEN_JOINT=True, p_offset_std=0.1, rpy_offset_std=0.01, axis_std=0.1, OUTPUT_NORMALIZE=False, seed=0, hdim=(16, 16), motor_embed_dim=4, lr=0.0015, lrd=0.95, wd=0.0, w_vec=0.001, epochs=500, focus_ratio=0.0, data_ratio=1.0, n_workers=2, batch_size=64, joint_seqs=('F', 'R', 'P', 'R', 'R', 'P', 'R'), marker_num=1, motor_dim=4)

In [7]:

from utils.dataloader import get_dataset, Sampler
train_dataset,val_dataset,test_dataset,ext_dataset  = get_dataset(f"../dataset/FINGER.json", 1.0)


train_sampler = Sampler(args, train_dataset)
val_sampler   = Sampler(args, val_dataset)
test_sampler  = Sampler(args, test_dataset)
ext_sampler   = Sampler(args, ext_dataset)


In [23]:
# %%
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.spatial.distance import pdist, cdist, squareform
from scipy.linalg import cholesky, cho_solve, solve_triangular

GPR_CHOLESKY_LOWER = True

class GPR(object):
    def __init__(self):
        self.alpha = 1e-2

    def se_kernel(self, Xa, Xb):
        # dists_ = pdist(Xa, metric='sqeuclidean')
        # K_ = np.exp(-.5 * gamma * dists_)
        # K_ = squareform(K_)
        # np.fill_diagonal(K, 1)

        dists = cdist(Xa, Xb, metric='sqeuclidean')
        K = np.exp(- dists / (self.median))

        # print(np.equal(K, K_))
        return K

    def fit(self, X, y):
        # Normalize fitting data
        self.X_mean = np.mean(X, axis=0)
        self.X_std = np.std(X, axis=0)
        self.X = (X-self.X_mean)/self.X_std

        self.y_mean = np.mean(y, axis=0)
        self.y_std = np.std(y, axis=0)
        self.y = (y-self.y_mean)/self.y_std

        # print(pdist(X, 'sqeuclidean'))
        self.median = np.median(pdist(X, 'sqeuclidean'))

        K = self.se_kernel(self.X, self.X)
        # noise™
        K[np.diag_indices_from(K)] += self.alpha
        try:
            self.L_ = cholesky(K, lower=GPR_CHOLESKY_LOWER, check_finite=False)
        except np.linalg.LinAlgError as exc:
            exc.args = (
                f"The kernel, {self.kernel_}, is not returning a positive "
                "definite matrix. Try gradually increasing the 'alpha' "
                "parameter of your GaussianProcessRegressor estimator.",
            ) + exc.args
            raise
        # Alg 2.1, page 19, line 3 -> alpha = L^T \ (L \ y)
        self.alpha_ = cho_solve(
            (self.L_, GPR_CHOLESKY_LOWER),
            self.y,
            check_finite=False,
        )
        return self
    
    def predict(self, X):
        X = (X-self.X_mean)/self.X_std
        K_trans = self.se_kernel(X, self.X)
        y_mean = K_trans @ self.alpha_
        y_mean = self.y_std * y_mean + self.y_mean
        # V = solve_triangular(
        #     self.L_, K_trans.T, lower=GPR_CHOLESKY_LOWER, check_finite=False
        # )
        
        return y_mean




from torch.nn import functional as F
def p_loss_fn(x,y):
    loss =  F.smooth_l1_loss(x,y, reduction='none')
    loss = torch.mean(loss, dim=list(range(1,loss.ndim)))
    return loss



# %%
### FK ###
batch = train_sampler.sample_all()
train_X, train_y = batch['motor_control'].numpy(), batch['position'].numpy()[:,0,:,0]


gpr = GPR()
gpr.fit(train_X, train_y)


batch = test_sampler.sample_all()

test_X, test_y = batch['motor_control'].numpy(), batch['position'].numpy()[:,0,:,0]


pred_y = gpr.predict(test_X)

import torch
a = torch.tensor(pred_y)
b = torch.tensor(test_y)
total_loss = p_loss_fn(a,b)

torch.mean(total_loss)


tensor(4.3868e-05, dtype=torch.float64)

(array([ 0.0041587 , -0.01215999,  0.10405478]),
 array([ 0.00335082, -0.01051756,  0.10437764], dtype=float32))